<div align="center">

# RIO Airbnb - Data Cleaning Jupyter Notebook 
**Latest Update:** _25th November 2022_

</div>

#### _Import required libraries_

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import OneHotEncoder

import airbnblib.cleaning as cln

#### *Import initial dataset*

In [2]:
listings_df = pd.read_csv("dataset/listings.csv", index_col=False)
listings_df.describe()

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,4.581500e+04,4.581500e+04,4.581500e+04,45802.000000,45802.000000,45815.000000,45815.000000,45815.000000,0.0,42055.000000,...,34968.000000,34961.000000,34967.000000,34958.000000,34957.000000,45815.000000,45815.000000,45815.000000,45815.000000,35236.000000
mean,1.641198e+17,2.022090e+13,1.496394e+08,86.630758,116.893957,33.990937,-118.239983,4.137400,NaN,1.864749,...,4.704121,4.840750,4.823367,4.798635,4.676207,22.099596,17.293878,4.399607,0.363789,1.577828
std,2.798853e+17,0.000000e+00,1.470969e+08,494.785019,640.543978,0.190502,0.236575,2.916169,NaN,1.224218,...,0.498639,0.410545,0.438933,0.407368,0.493803,81.801488,76.833890,30.262457,3.982576,1.894040
min,1.090000e+02,2.022090e+13,5.210000e+02,1.000000,1.000000,33.338480,-118.961880,0.000000,NaN,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.010000
25%,2.392476e+07,2.022090e+13,2.413248e+07,1.000000,2.000000,33.897055,-118.396360,2.000000,NaN,1.000000,...,4.640000,4.860000,4.830000,4.760000,4.600000,1.000000,1.000000,0.000000,0.000000,0.270000
50%,4.520569e+07,2.022090e+13,9.969611e+07,3.000000,4.000000,34.044060,-118.314920,4.000000,NaN,1.000000,...,4.860000,4.970000,4.970000,4.920000,4.810000,2.000000,1.000000,0.000000,0.000000,0.890000
75%,5.492515e+17,2.022090e+13,2.503126e+08,11.000000,18.000000,34.100115,-118.086906,6.000000,NaN,2.000000,...,5.000000,5.000000,5.000000,5.000000,4.960000,9.000000,5.000000,1.000000,0.000000,2.350000
max,7.118480e+17,2.022090e+13,4.786049e+08,4031.000000,20000.000000,34.822060,-117.627790,16.000000,NaN,24.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,638.000000,638.000000,342.000000,76.000000,86.210000


#### *Remove unnecessary data columns*

In [ ]:
REDUNDANT_COLUMNS = [
    'listing_url', 'scrape_id', 'last_scraped', 'source', 'host_id', 'host_url', 'host_name', 'host_location', 'host_thumbnail_url', 'host_picture_url', 'host_verifications', 
    'host_identity_verified', 'neighbourhood', 'property_type', 'bathrooms', 'calendar_updated', 'calendar_last_scraped'
]
for col in REDUNDANT_COLUMNS:
    listings_df.pop(col)   

#### *Fill in & replace selected value columns*

In [ ]:
PERCENTAGES = ['host_response_rate', "host_acceptance_rate"]
NULL_VALUES = {
    'license': 'none',
    'host_response_time': 'does not respond',
    'reviews_per_month': 0,
    'bathrooms_text': "0 private baths"
}

# Fill in null values in certain columns
for nul in NULL_VALUES:
    listings_df.loc[listings_df[nul].isnull(), nul] = NULL_VALUES[nul]

# Convert all values in 'license' to categorical values
listings_df.loc[listings_df["license"] == "Exempt", "license"] = "exempted"
listings_df.loc[~listings_df['license'].str.contains('none|exempted'), 'license'] = "have"

# Clean the 'price' column by removing the '$' and the ',' symbols as well as converting all values to "float" data type
listings_df['price'] = listings_df['price'].map(lambda x: float(x.lstrip('$').replace(",", "")))

#### *Convert values from `bathrooms_text` to individual columns of `bathroom_num (float)` and `bathroom_type (vectors)`*

In [ ]:
# Divide `bathrooms_text` column into `num` & `type`
NEW_COLUMNS = ['bathroom_type', 'bathroom_num']
for col in NEW_COLUMNS:
    listings_df[col] = listings_df["bathrooms_text"].map(lambda x: cln.bath_clean(x)[col])

listings_df = listings_df.join(pd.get_dummies(listings_df["bathroom_type"]))
listings_df.pop("bathrooms_text")

#### *Convert categorical columns to one-hot vectors*

In [ ]:
TRUE_FALSE = ['has_availability', 'instant_bookable', "host_is_superhost"]
ONE_HOT_VECTORS = ['license']

# Convert all t/f columns to true/false & add them to the ONE_HOT_VECTORS list
for tf in TRUE_FALSE:
    listings_df.loc[listings_df[tf] == 't', tf] = f"{tf}_true"
    listings_df.loc[listings_df[tf] == 'f', tf] = f"{tf}_false"
    ONE_HOT_VECTORS.append(tf)

# Convert all categorical columns into one-hot vectors
for column in ONE_HOT_VECTORS:
    listings_df = listings_df.join(pd.get_dummies(listings_df[column]))
    listings_df.pop(column)

#### *Remove all $0 listings and listings with z-score over +/-2.5*

In [ ]:
# Remove all listings with price of $0
listings_df = listings_df[listings_df['price'] != 0]

# Add a 'price_zscore' column
listings_df['price_zscore'] = stats.zscore(listings_df["price"])

# Remove all rows with a price z-score of more than +-2.5
listings_df = listings_df[abs(listings_df["price_zscore"]) < 2.5]

listings_df.describe()

#### *Convert all `amenities` rows to `list[str]` & eliminate all values within each amenities list that are less than 50% frequency*

In [ ]:
# For more documentation on these functions, check `airbnblib/cleaning.py`
listings_df['amenities'] = [cln.amenities_clean(row) for row in listings_df['amenities']]
listings_df['amenities'] = cln.amenities_freq(listings_df['amenities'])

#### *Extract listings zip code using using `latitude` and `longitude` columns*

In [4]:
# For more documentation on these functions, check `airbnblib/cleaning.py`
listings_df["address"] = [
    cln.get_zip_code(listings_df['latitude'].to_list()[i], listings_df['longitude'].to_list()[i])
    for i in range(45815)
]

<h2 align="center"><em>Tools for Referencing</em></h2>

#### **Export DataFrame to CSV**
Please close Excel or any other application that's opening the destination CSV before exporting, otherwise you'll get `[Errno 13] Permission Denied`

In [ ]:
listings_df.to_csv("dataset/cleaned_listings.csv", index=False)

#### **Check for unique variables in each column**

In [ ]:
col_name = input("Enter column name here: ")
display_list = []
for row in listings_df[col_name]: 
    if row not in display_list:
        display_list.append(row)

display_list

#### **Find percentage in of certain values**

In [ ]:
PERCENTAGES = {
    "sum": 0
}

listings_df.loc[listings_df["host_identity_verified"].isnull(), "host_identity_verified"] = "na"

for row in listings_df['host_identity_verified']:
    if row not in PERCENTAGES:
        PERCENTAGES[row] = 0
    PERCENTAGES[row] += 1
    PERCENTAGES["sum"] += 1
    
print("Percentage: " + str(
    PERCENTAGES["t"] / PERCENTAGES['sum']
))

#### **Find data type of certain columns**

In [ ]:
col_name = input("Enter column name here: ")
print(type(listings_df[col_name].to_list()[0]))